In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/deeplant

/content/drive/.shortcut-targets-by-id/1fjWczt1Ri5rqXEVYCJEVygNf_HhrXyax/deeplant


In [3]:
!pip install timm
!pip install --upgrade huggingface_hub
!pip install einops
!pip install mlflow
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:

In [4]:
import torch
import pandas as pd
import numpy as np
import timm
import os
import gc
import mlflow
import tqdm

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch import optim

import timm
import os
import gc

from torch import nn
from PIL import Image
from torch.utils.data import random_split, SubsetRandomSampler
from torch.utils.data import DataLoader

import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

#----------------For Making Model------------------------
import torch.nn.functional as F
from catboost import CatBoostRegressor
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


cuda


In [108]:
class CreateImageDataset(Dataset):
    def __init__(self, labels, img_dir, image_column, input_columns, output_columns, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.img_labels = labels
        self.image_column = image_column
        self.input_columns = input_columns
        self.output_columns = output_columns

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        output_label = torch.tensor(self.img_labels.iloc[idx, self.output_columns], dtype=torch.float32)
        input_label = torch.tensor(self.img_labels.iloc[idx, self.input_columns], dtype=torch.float32)

        name = self.img_labels.iloc[idx, self.image_column]
        img_path = os.path.join(self.img_dir, name)
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        input = {'image':image, 'input_label':input_label}
        return input, output_label, name


In [117]:
class LastModule(nn.Module):
    def __init__(self,params):
      super().__init__()
      # ( batch , 773 )
      input_shape = params['input_shape']
      # ( batch , 5 )
      output_shape = params['output_shape']
      self.fc1 = nn.Linear(input_shape, input_shape *4, bias=True)
      self.fc2 = nn.Linear(input_shape*4, input_shape*2, bias=True)
      self.fc3 = nn.Linear(input_shape * 2, output_shape, bias=True)
      self.dropout = nn.Dropout(0.3)

    def forward(self, x):
      x = F.gelu(self.fc1(x))
      x = self.dropout(F.gelu(self.fc2(x)))
      x = F.gelu(self.fc3(x))

      return x

In [ ]:
class VRModel(nn.Module):
  def __init__(self,params):
    super(VRModel,self).__init__()
    num_classes=params['num_classes']
    pretrained=params['pretrained']
    model_name=params['model_name']
    #--------------------vit-----------------------
    temp_model = timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes, exportable=True )
    features={'fc_norm':'out'}
    feature_extractor = create_feature_extractor(temp_model,return_nodes = features)
    vmodel = feature_extractor

    self.vision_model = vmodel

    self.numeric_model = CatBoostRegressor().load_model('catboost_mode.bin')

    #output layer
    self.lastfc_layer =  LastModule(params)

  def forward(self, image, num):

    image_output = self.vision_model(image)['out'] # [batch, 768]

    num = num.cpu().numpy()
    num_output = self.numeric_model.predict(num) # [batch, 5]
    num_output = torch.tensor(num_output, dtype=torch.float32).to(device)
    print(image_output.shape,num_output.shape)

    output = torch.cat([image_output, num_output], dim=-1) # [batch, num_classes * 2]
    output = self.lastfc_layer(output)

    return output
 # model = VRModel()
 # y=  model.forward()

In [ ]:
def regression(model, params):
    num_epochs=params['num_epochs']
    loss_func=params['loss_func']
    optimizer=params['optimizer']
    scheduler=params['lr_scheduler']
    train_dl=params['train_dl']
    val_dl=params['val_dl']
    sanity=params['sanity_check']
    num_classes=params['num_classes']
    columns_name=params['columns_name']

    train_loss, val_loss, train_metric, val_metric, train_acc, val_acc, r2_list =[], [], [], [], [], [], []
    best_loss = 0.0
    r2_score =0.0

    for epoch in (range(num_epochs)):

        #training
        model.train()
        loss, metric, _ = regression_epoch(model, loss_func, train_dl, epoch, num_classes, columns_name, sanity, optimizer)
        train_loss.append(loss)
        train_metric.append(metric)
        scheduler.step(train_loss[-1])

        #validation
        model.eval()
        with torch.no_grad():
            loss, metric, r2_score = regression_epoch(model, loss_func, val_dl, epoch, num_classes, columns_name, sanity)
        val_loss.append(loss)
        val_metric.append(metric)
        r2_list.append(r2_score)

        print('The Validation Loss is {} and the validation accuracy is {}'.format(val_loss[-1],val_metric[-1]))
        print('The Training Loss is {} and the training accuracy is {}'.format(train_loss[-1],train_metric[-1]))
        print('The Training Losses is {} and the validation losses is {}'.format(train_metric[-1],val_metric[-1]))
        print('The R2 score(fixed) is {}'.format(r2_score))

    return model, train_metric, val_metric, train_loss, val_loss, r2_list


# calculate the loss per epochs
def regression_epoch(model, loss_func, dataset_dl, epoch, num_classes, columns_name, sanity_check=False, opt=None):
    running_loss = 0.0
    running_metrics = np.zeros(num_classes)
    running_y = None
    running_output = None
    len_data = len(dataset_dl.dataset)
    r2_score =0.0
    #-------------결과 저장할 data frame 정의하는 곳-------------
    # 여기 바꾸면 아래 validation 저장하는 부분도 바꿔야함.
    columns = ['file_name']
    for i in range(num_classes):
        columns.append('predict ' + columns_name[i])
        columns.append('label ' + columns_name[i])
    df = pd.DataFrame(columns=columns)
    #----------------------------------------------------------

    for xb, yb, name_b in (dataset_dl):
        imageb = xb['image'].to(device)
        inputb = xb['input_label'].to(device)
        yb = yb.to(device)
        output = model(imageb,inputb)

        metric_b = np.zeros(num_classes)
        total_loss = 0.0

        # class가 1개일 때 개별 라벨이 list 형식아니라서 for문을 못 돌림. 그래서 일단 구분함.
        if num_classes != 1:
            for i in range(num_classes):
                loss_b = loss_func(output[:, i], yb[:, i])
                total_loss += loss_b
                metric_b[i] += loss_b.item()
        else:
            loss_b = loss_func(output, yb)
            total_loss += loss_b
            metric_b += loss_b.item()

        if opt is not None:
            opt.zero_grad()
            total_loss.backward()
            opt.step()

        if opt is None:
            #-------------------- validation값 저장하는 부분 -------------------------
            # 여기 바꾸면 위에 data frame 정의하는 곳도 바꿔야함.
            output = output.detach().cpu().numpy()
            yb = yb.cpu().numpy()

            if running_y is None:
                running_y = np.array(yb)
            else:
                running_y = np.vstack((running_y,yb))

            if running_output is None:
                running_output = np.array(output)
            else:
                running_output = np.vstack((running_output,output))

            output = list(output)
            yb = list(yb)
            name_b = list(name_b)
            for i in range(len(output)):
                data = {'file_name':name_b[i]}
                # class 개수 1개면 문제 생겨서 나눔.
                if num_classes != 1:
                    for j in range(num_classes):
                        data['predict ' + columns_name[j]] = output[i][j]
                        data['label ' + columns_name[j]] = yb[i][j]
                else:
                    data['predict ' + columns_name[0]] = output[i]
                    data['label ' + columns_name[0]] = yb[i]
                new_row = pd.DataFrame(data=data, index=['file_name'])
                df = pd.concat([df,new_row], ignore_index=True)


            if not os.path.exists('model2_output'):
                os.mkdir('model2_output')
            df.to_csv(f'model2_output/output_{epoch}.csv')
            #------------------------------------------------------------------------

        running_loss += total_loss.item()
        if metric_b is not None:
            running_metrics += metric_b

        if sanity_check is True:
            break

    if opt is None:
        n = len(running_y)
        print("n:",n)
        y_mean = running_y.mean(axis=0)
        print("y_mean:",y_mean)
        ssr = np.square(running_y - running_output).sum(axis=0)
        print("ssr:",ssr)
        sst = np.square(running_y - y_mean).sum(axis=0)
        print("sst:",sst)
        print("1 - (ssr / sst):",1 - (ssr / sst))
        r2_score = (1 - (ssr / sst)).mean()
        print("r2_score:",r2_score)

    loss = running_loss / len_data
    metric = running_metrics / len_data
    return loss, metric, r2_score


In [ ]:
imagepath = '/content/drive/MyDrive/deeplant/data/train'
label_set = pd.read_csv('/content/drive/MyDrive/deeplant/model/raw_csv/train.csv')
image_column = 13
input_columns = [14,15,16,17]
output_columns = [8,9,10,11,12]

In [ ]:
#-----------Define input transform------------
transformation = transforms.Compose([
transforms.Resize([448,448]),
transforms.RandomHorizontalFlip(p=0.3),
transforms.RandomVerticalFlip(p=0.3),
transforms.RandomRotation((-20,20)),
transforms.ToTensor(),
])
#---------------------------------------------------
loss_func = nn.MSELoss()
train_dataset = CreateImageDataset(label_set, imagepath, image_column, input_columns, output_columns, transform=transformation)
test_dataset = CreateImageDataset(label_set, imagepath, image_column, input_columns, output_columns, transform=transformation)
#-------------------------------------------------------

params_model = {
    'model_name':'vit_base_patch32_clip_448.laion2b_ft_in12k_in1k',
    'pretrained':True,
    'num_classes':5,
    'input_shape':773,
    'output_shape':5,
}
#-------------------------------------------------------
model = VRModel(params_model)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr = 1e-5)
scheduler = ReduceLROnPlateau(optimizer, patience = 2, factor = 0.5, threshold = 0.003)
train_dl = DataLoader(train_dataset, batch_size=8, num_workers=0, pin_memory=True)
val_dl = DataLoader(test_dataset, batch_size=8, num_workers=0, pin_memory=True)
#----------------------------------------------------

In [ ]:
params_train = {
'num_epochs':30,
'optimizer':optimizer,
'loss_func':loss_func,
'train_dl':train_dl,
'val_dl':val_dl,
'sanity_check':False,
'lr_scheduler':scheduler,
'num_classes':5,
'columns_name':label_set.columns[output_columns].values,
}

In [ ]:
model, train_acc, val_acc, train_loss, val_loss, r2_score = regression(model, params_train)

#plot the curves
plt.plot(train_acc, label = 'train_acc')
plt.plot(val_acc, label = 'val_acc')
plt.plot(train_loss, label = 'train_loss')
plt.plot(val_loss, label = 'val_loss')
plt.legend()
plt.title('Accuracy and Loss Plots')
figure = plt.gcf()
plt.clf()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torch.Size([8, 768]) torch.Size([8, 5])
torc

In [ ]:
model.cpu()
del model
gc.collect()
torch.cuda.empty_cache()

# 실험실

In [ ]:
model = CatBoostRegressor()

In [ ]:
model.ge

In [98]:
vision_model

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): 

In [96]:
vision_model = timm.create_model('vit_base_patch32_clip_448.laion2b_ft_in12k_in1k', pretrained=True, num_classes=5, exportable=True )

In [97]:
node, _ =get_graph_node_names(vision_model)
node

['x',
 'patch_embed.getattr',
 'patch_embed.getitem',
 'patch_embed.getitem_1',
 'patch_embed.getitem_2',
 'patch_embed.getitem_3',
 'patch_embed.eq',
 'patch_embed._assert',
 'patch_embed.eq_1',
 'patch_embed._assert_1',
 'patch_embed.proj',
 'patch_embed.flatten',
 'patch_embed.transpose',
 'patch_embed.norm',
 'cls_token',
 'getattr',
 'getitem',
 'expand',
 'cat',
 'pos_embed',
 'add',
 'pos_drop',
 'patch_drop',
 'norm_pre',
 'blocks.0.norm1',
 'blocks.0.attn.getattr',
 'blocks.0.attn.getitem',
 'blocks.0.attn.getitem_1',
 'blocks.0.attn.getitem_2',
 'blocks.0.attn.qkv',
 'blocks.0.attn.reshape',
 'blocks.0.attn.permute',
 'blocks.0.attn.unbind',
 'blocks.0.attn.getitem_3',
 'blocks.0.attn.getitem_4',
 'blocks.0.attn.getitem_5',
 'blocks.0.attn.q_norm',
 'blocks.0.attn.k_norm',
 'blocks.0.attn.mul',
 'blocks.0.attn.transpose',
 'blocks.0.attn.matmul',
 'blocks.0.attn.softmax',
 'blocks.0.attn.attn_drop',
 'blocks.0.attn.matmul_1',
 'blocks.0.attn.transpose_1',
 'blocks.0.attn.resh

In [ ]:
features={'norm':'out'}
feature_extractor = create_feature_extractor(vision_model,return_nodes = features)
new_model = feature_extractor

In [ ]:

F.gelu()

TypeError: ignored